# 유사한 아티스트 추천 시스템

유저가 좋아하는 특정 아티스트와 유사한 다른 아티스트를 추천하는 추천 시스템
- [Last.fm](https://www.last.fm/)에서는 어떤 유저가 특정 아티스트의 노래를 몇 번이나 들었는지에 대한 데이터를 제공
- 2010년에 미국에서 서비스되고 있는 spotify에서 발생한 데이터

<br>

## 1. 데이터 탐색 및 전처리

### 1.1 &nbsp;데이터 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

fname = '/content/drive/MyDrive/DS/Recommender System/rec_data/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의의 컬럼명 지정
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있음
data.head(10)
data

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
...,...,...,...,...
17535650,"sep 20, 2008",7ffd711a-b34d-4739-8aab-25e045c246da,turbostaat,12
17535651,"sep 20, 2008",9201190d-409f-426b-9339-9bd7492443e2,cuba missouri,11
17535652,"sep 20, 2008",e7cf7ff9-ed2f-4315-aca8-bcbd3b2bfa71,little man tate,11
17535653,"sep 20, 2008",f6f2326f-6b25-4170-b89d-e235b25508e8,sigur rós,10


In [ ]:
# 사용하는 컬럼만 남기기
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [ ]:
# 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 변환
data['artist'] = data['artist'].str.lower()
data.head(10)

/tmp/ipython-input-5-2617234066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['artist'] = data['artist'].str.lower()


,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [ ]:
# 첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


생소한 아티스트가 많은 것을 볼 수 있다.

이처럼 추천 시스템에서는 모르면 검증을 못 하기 때문에, 적용하는 분야에 대한 지식이 특히 더 필요하다.

<br>

### 1.2 &nbsp;데이터 탐색

추천 모델을 만들기 전에 데이터의 기본적인 정보 확인하기

- 유저 수, 아티스트 수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
- 유저 play 횟수 중앙값에 대한 통계

In [ ]:
# 유저 수
data['user_id'].nunique()

358868

In [ ]:
# 아티스트 수
data['artist'].nunique()

291346

In [ ]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

,user_id
artist,
radiohead,77254
the beatles,76245
coldplay,66658
red hot chili peppers,48924
muse,46954
metallica,45233
pink floyd,44443
the killers,41229
linkin park,39773


In [ ]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

,artist
count,358868.000000
mean,48.863234
std,8.524272
min,1.000000
25%,46.000000
50%,49.000000
75%,51.000000
max,166.000000


In [ ]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

,play
count,358868.000000
mean,142.187676
std,213.089902
min,1.000000
25%,32.000000
50%,83.000000
75%,180.000000
max,50142.000000


<br>

### 1.3 &nbsp;모델 검증을 위한 사용자 초기 정보 세팅

우리는 본인의 음악 취향과 가장 유사한 아티스트를 추천받고 싶은데,

유튜브 뮤직 등 추천 시스템들은 이를 위해서 처음 가입하는 사용자의 취향과 유사한 아티스트 정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많다.

만든 추천 시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 한다.

아래와 같이 좋아하는 유명한 아티스트들을 기존의 데이터에 추가해서 넣도록 한다.

In [ ]:
# 좋아하는 아티스트 리스트
my_favorite = ['red hot chili peppers' , 'radiohead' ,'muse' ,'metallica' ,'nirvana']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = pd.concat([data, my_playlist])                            # 위에 임의로 만든 my_playlist 데이터를 추가해 줌

data.tail(10)  # 잘 추가되었는지 확인

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,red hot chili peppers,30
1,zimin,radiohead,30
2,zimin,muse,30
3,zimin,metallica,30
4,zimin,nirvana,30


<br>

### 1.4 &nbsp;모델에 활용하기 위한 전처리

사람이 태어나면 주민등록번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주기

- 해당 데이터에서는 user와 artist 각각에 번호를 붙이고 싶은데
- 보통 이런 작업을 indexing이라고 함


In [ ]:
# 고유한 유저, 아티스트를 찾기
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하기
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [ ]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['zimin'])  # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 함
print(artist_to_idx['black eyed peas'])

358868
376


In [ ]:
# indexing을 통해 데이터 컬럼 내 값 변환

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series 구하기
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,5,30
1,358868,217,30
2,358868,277,30
3,358868,283,30


data의 `user_id`와 `artist` 컬럼 내 값들이 모두 정수 인덱스 값으로 변경된 것을 확인하였다.  
이것으로 훈련을 위한 전처리가 완료되었다.

<br>

### 1.5 &nbsp;사용자의 명시적/암묵적 평가

추천 시스템은 사용자들이 아이템을 얼마나 선호하는지를 모델링 하기를 원한다. 이를 위해서는 사용자의 아이템 선호도를 말해 주는 유저 행동 데이터셋이 필요하다.

해당 데이터는 사용자가 아티스트의 곡을 몇 번 플레이했나 하는 것뿐인데, 이렇게 서비스를 사용하면서 자연스럽게 발생하는 암묵적(implicit)인 피드백도 사용자의 아이템에 대한 평가를 알 수 있는 단서가 될 수 있다.

[Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf) 논문에는 추천시스템에서 암묵적 피드백 데이터셋을 활용할 때의 고민이 잘 담겨있고, 이를 위해 염두해두어야할 암묵적 피드백 데이터셋의 특징을 다음과 같이 정리하였다.

- 부정적인 피드백이 없다.(No Negative Feedback)
- 애초에 잡음이 많다.(Inherently Noisy)
- 수치는 신뢰도를 의미한다.(The numerical value of implicit feedback indicates confidence)
- Implicit-feedback Recommender System의 평가는 적절한 방법을 고민해봐야 한다.(Evaluation of implicit-feedback recommender requires appropriate measures)

<br>

Q. 어떤 유저가 아티스트의 곡을 한 번만 들었다는 것의 의미?  
    A. 명확한 정답은 존재하지 않는다. 어떤 유저는 한 번 듣고 별로라고 생각했을 수도 있고, 어떤 유저는 다시 듣고 싶지만 가수의 이름을 잊어버렸을 수도 있다. 이런 애매한 암묵적 데이터야말로 도메인 지식과 직관이 활용되어야 하는 영역이다.



In [ ]:
# 1회만 play한 데이터의 비율 확인하기
only_one = data[data['play']==1]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147739,17535660
Ratio of only_one over all data is 0.84%


해당 프로젝트에서 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용

1. 한 번이라도 들었으면 선호한다고 판단한다.

2. 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

<br>
<br>

## 2. Matrix Factorization(MF)

### 2.1 &nbsp;CSR(Compressed Sparse Row) Matrix

In [ ]:
# data를 CSR Matrix에 맞게 변형하기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 17535578 stored elements and shape (358869, 291347)>

In [ ]:
csr_data.shape

(358869, 291347)

<br>

### 2.2 &nbsp;MF 모델 학습하기

In [ ]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 73.3 MB/s eta 0:00:00


In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
"""
factors: 유저와 아이템의 벡터를 몇 차원으로 할 것인지
regularization: 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
use_gpu: GPU를 사용할 것인지
iterations: 데이터를 몇 번 반복해서 학습할 것인지 (epochs와 같은 의미)
 -> factors, iterations를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 적절한 값을 찾아야 함
"""
# Implicit AlternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(
    factors=100,
    regularization=0.01,
    use_gpu=False,
    iterations=15,
    dtype=np.float32
)

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [ ]:
# 모델 훈련
als_model.fit(csr_data)

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
print("user_factors shape:", als_model.user_factors.shape)
print("item_factors shape:", als_model.item_factors.shape)

user_factors shape: (358869, 100)
item_factors shape: (291347, 100)


<br>

모델 학습이 끝났고, 아래의 2가지 사항 살펴보기

1. Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지

In [ ]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

In [ ]:
zimin_vector

array([-0.43977648,  0.7469346 , -0.0978633 ,  0.16884415, -0.02504694,
        0.7681467 ,  0.949022  ,  0.64858836, -0.46911892, -2.788568  ,
       -1.2178308 , -1.0228878 ,  0.07740878, -0.37443632,  0.03789837,
        1.2697114 ,  0.4900352 , -0.9080462 , -0.49530682, -0.12416363,
        0.21079755,  1.2780226 ,  0.29346094,  0.58726674,  1.3837903 ,
        0.87075865, -0.9920229 ,  0.0165251 ,  0.39895317, -0.12109733,
       -0.47901198,  1.049176  , -0.02725262,  0.4036412 , -0.8883452 ,
       -0.7202034 , -1.8047192 ,  1.021843  , -1.806024  , -0.13902089,
        0.49179256, -1.3023041 ,  1.5945412 , -0.46959037, -0.07811864,
       -0.16900352,  0.2619719 ,  1.0341628 , -1.2016938 ,  1.700595  ,
        1.2622094 , -0.06703284,  0.16185857,  1.4581709 , -1.4452509 ,
        0.6404994 , -0.07799183,  0.7373444 ,  0.92957747, -0.50510067,
        0.79455525, -0.18437362, -0.98569554,  1.8877704 ,  1.2654274 ,
       -0.32108733, -0.4984242 , -0.92082375, -1.4217596 , -0.46

In [ ]:
black_eyed_peas_vector

array([-0.00666112, -0.00451628, -0.02030989,  0.00553202,  0.00098537,
        0.01108464,  0.01744452,  0.00960084,  0.00639869,  0.0078463 ,
        0.02228912, -0.00412865,  0.01306266,  0.00252362,  0.02312126,
        0.01680857,  0.02529267,  0.02576204,  0.00666401,  0.0075205 ,
        0.00741739,  0.00355046, -0.01578015, -0.0029534 ,  0.00914012,
        0.00176658,  0.00772388,  0.00381269, -0.01589671,  0.0003708 ,
        0.00993262,  0.00673008,  0.02997865,  0.01685858,  0.00211645,
        0.00782852, -0.00549743,  0.01914245, -0.0012684 ,  0.00985622,
        0.01608234,  0.00421241,  0.00611397, -0.00259477, -0.00293772,
        0.03157257,  0.01515277,  0.01102153, -0.0008465 ,  0.00700587,
        0.01065515, -0.00352005,  0.00969111, -0.00212505,  0.0036836 ,
        0.00623417, -0.00171795,  0.0068805 , -0.00212306,  0.02987833,
       -0.00514591,  0.02084196, -0.00619149,  0.0237116 ,  0.02091092,
        0.00748867,  0.00067659,  0.01372784, -0.00066499,  0.00

In [ ]:
# zimin과 black_eyed_peas를 내적
np.dot(zimin_vector, black_eyed_peas_vector)

np.float32(0.14819819)

In [ ]:
# 모델이 zimin의 queen에 대한 선호도 예측 확인하기
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

np.float32(0.5923497)

<br>
<br>

## 3. 유사한 아티스트 추천하기

### 3.1 &nbsp;유사한 아티스트 찾기

In [ ]:
# clodplay와 유사한 아티스트 찾기
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

(array([  62,  277,   28,    5,  217,  473,  490,  247,  418,  694,  910,
         268, 1018,   55,  782], dtype=int32),
 array([1.        , 0.9871467 , 0.98140043, 0.9746498 , 0.96875894,
        0.96622646, 0.9613367 , 0.9593878 , 0.94662964, 0.9448347 ,
        0.94365704, 0.9329869 , 0.9321372 , 0.92701954, 0.92388606],
       dtype=float32))

(아티스트의 id, 유사도) Tuple 로 반환하고 있다.  
아티스트의 id를 다시 아티스트의 이름으로 매핑 시켜야 한다.

In [ ]:
# artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay', 'die ärzte']

In [ ]:
# 반복 및 확인을 위해 위의 로직 함수화
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i] for i in similar_artist[0]]
    return similar_artist

In [ ]:
# 2pac과 유사한 아티스트 찾기
get_similar_artist('2pac')

['2pac',
 'notorious b.i.g.',
 'nas',
 'dr. dre',
 'the game',
 'snoop dogg',
 '50 cent',
 'jay-z',
 'dmx',
 'ice cube']

마니아들은 특정 장르의 아티스트들에게로 선호도가 집중되고, 다른 장르의 아티스트들과는 선호도가 낮게 나타날 것이다.  
이런 마니아들의 존재로 인해 같은 장르의 아티스트들의 벡터들도 더 가까워져서 `get_similar_artist()`에서 장르별 특성이 두드러지게 될 것이다.

In [ ]:
# lady gaga와 유사한 아티스트 찾기
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'kelly clarkson',
 'christina aguilera',
 'leona lewis',
 'mariah carey']

<br>

### 3.2 &nbsp;유저에게 아티스트 추천하기


AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통해 유저가 좋아할 만한 아티스트를 추천
- filter_already_liked_items: 유저가 이미 평가한 아이템은 제외

In [ ]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받음
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=False)
artist_recommended

(array([ 247,    5,  910,  277,   62,  268,  217,  473,   75,  418,   28,
         694,  773,  531,  279,  193,  490,  283, 1018,  776], dtype=int32),
 array([0.6626629 , 0.6531961 , 0.6439305 , 0.64247304, 0.6420031 ,
        0.6229397 , 0.62029976, 0.5936072 , 0.5923497 , 0.591262  ,
        0.5757414 , 0.57288235, 0.5707892 , 0.5599797 , 0.5553293 ,
        0.5529357 , 0.54928696, 0.54747975, 0.5445354 , 0.54288745],
       dtype=float32))

In [ ]:
[idx_to_artist[i] for i in artist_recommended[0]]

['the beatles',
 'red hot chili peppers',
 'nirvana',
 'muse',
 'coldplay',
 'pink floyd',
 'radiohead',
 'placebo',
 'queen',
 'u2',
 'the killers',
 'foo fighters',
 'nine inch nails',
 'depeche mode',
 'led zeppelin',
 'system of a down',
 'oasis',
 'metallica',
 'the smashing pumpkins',
 'the cure']

AlternatingLeastSquares 클래스에 구현된 explain 메서드를 사용하면  
유저가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.
- 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됨)를 반환

In [ ]:
oasis = artist_to_idx['oasis']
explain = als_model.explain(user, csr_data, itemid=oasis)

In [ ]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('muse', np.float64(0.1252568780458359)),
 ('radiohead', np.float64(0.12398672241769026)),
 ('red hot chili peppers', np.float64(0.12154230899306624)),
 ('nirvana', np.float64(0.11142605988236935)),
 ('metallica', np.float64(0.07098007074843088))]

<br>

지금까지 추천 시스템에서 Baseline으로 많이 사용되는 MF를 통해 유저에게 아티스트를 추천해 보았다.

그러나 만든 모델은 몇 가지 아쉬운 점이 있다.

유저, 아티스트에 대한 Meta 정보를 반영하기 쉽지 않음. 연령대별로 음악 취향이 다를 것이며, 유저가 언제 play 했는지에 반영하기 쉽지 않다.

이러한 이유와 딥러닝의 발전으로 MF 이외의 모델 구조도 많이 연구/ 사용되고 있지만, 어떤 추천 모델도 핵심은 MF와 비슷하다.

유저와 아이템에 대한 벡터를 잘 학습하여 취향에 맞게(유저에 맞게) 아이템을 보여주거나(Retrieval) 걸러내는(Filtering) 역할이다.